<a href="https://colab.research.google.com/github/afoteygh/Research/blob/main/GPT2_%20774M_Snort_Rule_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing Dependencies
Run the cell below to install model dependencies

In [ ]:
from datetime import datetime

In [ ]:
# having problems with tf 2.0+

%tensorflow_version 1.x
!git clone https://github.com/Akings/snort_rule_gen.git
#!git clone https://github.com/afoteygh/SnortRuleGen.git
!pip install -r /content/snort_rule_gen/requirements.txt
from datetime import datetime
from google.colab import files

TensorFlow 1.x selected.
Cloning into 'snort_rule_gen'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 56 (delta 23), reused 30 (delta 9), pack-reused 0
Unpacking objects: 100% (56/56), done.
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=e50b7eee0ba7a60e10720cd8472e24112ee9db57226ee87a9c23aa71052a6cc0
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3


In [ ]:
!pip install wandb

     |████████████████████████████████| 2.0MB 13.7MB/s 
     |████████████████████████████████| 133kB 55.8MB/s 
     |████████████████████████████████| 102kB 13.9MB/s 
     |████████████████████████████████| 163kB 55.5MB/s 
     |████████████████████████████████| 71kB 10.2MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6490 sha256=6a84a02c28baa5daa611d67503d0564af6ab34c7f5b11addc2ec6158a860a8a7
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp36-none-any.whl size=8785 sha256=5add86dbabde6bace1205c51975f72109bc91cac9bfaaf4a32cbd84df1e3bac9
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools


In [ ]:
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
import wandb
wandb.init(project="gpt2-117mdefault")

wandb: Currently logged in as: afotey (use `wandb login --relogin` to force relogin)


In [ ]:
import tarfile
import os
import json
import requests
import sys
import shutil
import re
from tqdm import tqdm, trange
import numpy as np
import tensorflow as tf
from tensorflow.core.protobuf import rewriter_config_pb2
from tensorflow.python.client import device_lib
from nltk.translate.bleu_score import SmoothingFunction, corpus_bleu, sentence_bleu
import time
import csv
import argparse

Import our utilities

In [ ]:
from snort_rule_gen.utilities import model, sample, encoder, memory_saving_gradients
from snort_rule_gen.utilities.load_dataset import load_dataset, Sampler
from snort_rule_gen.utilities.accumulate import AccumulatingOptimizer

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



function to download files from the internet with nice progress bar 

Import model code here

In [ ]:
from snort_rule_gen import srg

To Monitor the colab

In [ ]:
from urllib.request import urlopen


In [ ]:
exec(urlopen("http://colab-monitor.smankusors.com/track.py").read())
_colabMonitor = ColabMonitor().start()

In [ ]:
srg.download_vocab(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 580Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 108Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 681Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:07, 66.1Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 393Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 194Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 234Mit/s]                                                       


## Mounting Google Drive

The best way to get input text to-be-trained into the Colab VM, and to get the trained model out of Colab, is to route it through Google Drive *first*.

Running this cell will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code)

In [ ]:
srg.mount_gdrive()

Mounted at /content/drive


## Uploading Rules File to be Trained
In the Colab sidebar on the left of the screen, select the folder icon. From there select dirve then  My Drive then you can upload files there
 Then run the cells below to copy file into colab

In [ ]:
file_name = "emerging-trojanOriginal.txt" #  filename of your file

In [ ]:
srg.copy_file_from_gdrive(file_name)

## Training config

The next cell creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `finetuning`:


*  **`restore_from`**: Set to `fresh` to start training from the base model, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
 

In [ ]:
sess = srg.start_tf_sess()

srg.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=10000,
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              sample_every=200,
              save_every=500
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


100%|██████████| 1/1 [00:00<00:00,  6.02it/s]

Loading dataset...
dataset has 33759 tokens
Training...


[10 | 13.85] loss=1.70 avg=1.70
[20 | 21.27] loss=1.20 avg=1.45
[30 | 28.69] loss=1.20 avg=1.36
[40 | 36.12] loss=1.00 avg=1.27
[50 | 43.56] loss=0.84 avg=1.18
[60 | 51.00] loss=0.63 avg=1.09
[70 | 58.43] loss=0.52 avg=1.00
[80 | 65.86] loss=0.57 avg=0.95
[90 | 73.28] loss=0.37 avg=0.88
[100 | 80.72] loss=0.32 avg=0.82
[110 | 88.15] loss=0.35 avg=0.78
[120 | 95.57] loss=0.19 avg=0.72
[130 | 103.01] loss=0.21 avg=0.68
[140 | 110.42] loss=0.13 avg=0.64
[150 | 117.85] loss=0.14 avg=0.60
[160 | 125.28] loss=0.12 avg=0.57
[170 | 132.72] loss=0.10 avg=0.54
[180 | 140.14] loss=0.09 avg=0.51
[190 | 147.56] loss=0.07 avg=0.49
[200 | 154.99] loss=0.07 avg=0.47
======== SAMPLE 1 ========
/dst; content:"|0d 0a|Content-Length|2e 39 2a|"; fast_pattern:only; http_header; reference:url,damballa.com/enterprise/tag-whitepapers/2001101/damballa-gets-its-day-out-with-its-bad-mocha/; reference:url,securitykitten.com/entry/attempts-to-detect-damballa-transactions-and-transactions-using-junk DNS CnC Domain|3

After the model is trained, you can copy the checkpoint folder to your own Google Drive if you want to use it later.

If you want to download it to your personal computer, I recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally for use.

if you already have a checkpoint go to train model from checkpoint

In [ ]:
srg.copy_checkpoint_to_gdrive(run_name='run1')

OSError: ignored

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.tar` checkpoint file from your Google Drive into Colab

the filename should match `run1` in the celll below

In [ ]:
srg.copy_checkpoint_from_gdrive(run_name='run1')

sess = srg.start_tf_sess()
srg.load_gpt2(sess, run_name='run1')

FileNotFoundError: ignored

## Generate Rules From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate rules. `generate` generates a single rules from the loaded model.

This is a simple test for a small number of rules just to explain some helpful parmeters

In [ ]:
srg.generate(sess, run_name='run1')

classtype:trojan-activity; sid:2012266; rev:1; metadata:created_at 2011_04_04, updated_at 2019_09_28;)


alert tcp $HOME_NET any -> $EXTERNAL_NET $HTTP_PORTS (msg:"ET TROJAN Possible Drive DDoS Check-in"; flow:established,to_server;  content:"k="; fast_pattern; http_client_body; depth:2; content:"POST"; http_method; content:!"Referer|3a|"; http_header; content:"Content-Length|3a 20|17|0d 0a|"; http_header; content:"Host|3a|"; depth:5; http_header; pcre:"/-urlencoded\r\n(\r\n)?$/H"; pcre:"/^k=[a-z0-9]{15}$/P"; pcre:"/^k=[0-9]*?[a-z]/P"; flowbits:set,ET.Drive.DDoS.Checkin; classtype:trojan-activity; sid:2012269; rev:1; metadata:created_at 2011_04_06, updated_at 2019_09_28;)


alert tcp $HOME_NET any -> $EXTERNAL_NET $HTTP_PORTS (msg:"ET TROJAN CBReplay Checkin"; flow:established,to_server; content:"POST"; http_method; content:"/index.php"; http_uri; urilen:10; content:"filename="; http_client_body; content:!"Referer|3a|"; http_header; content:!"User-Agent|3a|"; http_header; pcre:"/^Host\

If you're creating an API based on your model and need to pass the generated rules elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the rules to start with a given character sequence and generate rules from there (good if you want to know where the rule starts).

You can also generate multiple rules at a time by specifing `nsamples`. you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (set a maximum of 20 for `batch_size` to avoid running out of memory).

Other optional-but-helpful parameters

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the output (default 0.7, please keep it between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)

See how they are used below

In [ ]:
srg.generate(sess,
              length=250,
              top_p=0.9,
              temperature=1.0,
              prefix="alert tcp",
              nsamples=5,
              batch_size=5
              )

alert tcp $HOME_NET any -> $EXTERNAL_NET $HTTP_PORTS (msg:"ET TROJAN TDSS/TDL/Alureon MBR rootkit Checkin"; flow:established,to_server; urilen:16<>402; content:"GET"; nocase; http_method; content:"Accept-Language|3a| "; http_header; depth:17; content:!"Accept|3a| "; http_header; content:"User-Agent|3a| Mozilla/4.0 |28|compatible|3b| MSIE"; fast_pattern:23,18; http_header; content:"Host|3a| "; distance:0; http_header; content:"|3a| no-cache"; distance:0; http_header; pcre:"/^\/[a-z0-9+\/=]{16,400}$/Ui"; metadata: former_category MALWARE; classtype:trojan-activity; sid:2011894; rev:18; metadata:created_at 2010_11_05, updated_at 2010_11_05
alert tcp $EXTERNAL_NET $HTTP_PORTS -> $HOME_NET any (msg:"ET WEB_CLIENT Hex Obfuscation of unescape % Encoding"; flow:established,to_client; content:"%75%6e%65%73%63%61%70%65"; nocase; fast_pattern:only; reference:url,cansecwest.com/slides07/csw07-nazario.pdf; reference:url,www.sophos.com/security/technical-papers/malware_with_your_mocha.html; classtyp

# Bulk Generation

For bulk generation, you can generate a large amounts rules to a `.txt` file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [ ]:
gen_file = 'snort_genrule_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

srg.generate_to_file(sess,
                      prefix="alert tcp",
                      destination_path=gen_file,
                      length=500,
                      top_p=0.9,
                      temperature=1.0,
                      nsamples=500,
                      batch_size=20,
                      )

In [ ]:
# to download the file
files.download(gen_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Evaluation

In [ ]:
!git clone https://github.com/google-research/bleurt.git


fatal: destination path 'bleurt' already exists and is not an empty directory.


In [ ]:
!cd /content/bleurt
!ls
!pip install  /content/bleurt

bleurt		     drive				snort_rule_gen
checkpoint	     sample_data
checkpoint_run1.tar  snort_genrule_20210202_021309.txt
Processing ./bleurt
     |████████████████████████████████| 358kB 7.9MB/s 
  Created wheel for BLEURT: filename=BLEURT-0.0.1-cp36-none-any.whl size=60867 sha256=26d10d653e3950aac5475ba8fcdd4578a14383472b9bfa4462086c6126168df1
  Stored in directory: /tmp/pip-ephem-wheel-cache-1795gqr1/wheels/7b/78/21/308004964c291bd9971f3a886b6219e9a7873a8fe4b2fefc10
Successfully built BLEURT


In [ ]:
tf.app.flags.DEFINE_string('f', '', 'kernel')

##**BLEU**

In [ ]:
from bleurt import score

checkpoint = "/content/bleurt/bleurt/test_checkpoint"
references = ['alert tcp $HOME_NET any -> $EXTERNAL_NET 1024:65535 (msg:"ET TROJAN Bandook v1.35 Socks5 Proxy Start Command Reply"; flowbits:isset,BE.Bandook1.35; flow:established,to_server; dsize:10; content:"|9a 86 8a 82 9a 86 87 26 26 26|"; offset:0; reference:url,www.nuclearwintercrew.com; reference:url,research.sunbelt-software.com/threatdisplay.aspx?name=Bandook&threatid=40408; reference:url,doc.emergingthreats.net/bin/view/Main/TrojanBandook; classtype:trojan-activity; sid:2003564; rev:5; metadata:created_at 2010_07_30, updated_at 2010_07_30;)','alert tcp $HOME_NET any -> $EXTERNAL_NET 25 (msg:"ET TROJAN elitekeylogger v1.0 reporting - Outbound"; flow:established,to_server; content:"MAIL FROM|3a|<logs@logs.com>"; reference:url,doc.emergingthreats.net/2002941; classtype:trojan-activity; sid:2002941; rev:5; metadata:created_at 2010_07_30, updated_at 2010_07_30;)','alert tcp $HOME_NET any -> $EXTERNAL_NET $HTTP_PORTS (msg:"ET TROJAN Drop.Agent.bfsv HTTP Activity (UsER-AgENt)"; flow:established,to_server; content:"GeT"; http_method; content:"HttP"; depth:200; content:"|0d 0a|HoST|3a| "; content:"UsER-AgENt|3a| |0d 0a|"; http_header; metadata: former_category MALWARE; reference:url,doc.emergingthreats.net/2010129; classtype:trojan-activity; sid:2010129; rev:6; metadata:created_at 2010_07_30, updated_at 2020_02_25;)']
candidates = ['alert tcp $HOME_NET any -> $EXTERNAL_NET $HTTP_PORTS (msg:"ET TROJAN Jiff Ransomware Checkin"; flow:established,to_server; content','alert tcp $HOME_NET any -> $EXTERNAL_NET $HTTP_PORTS (msg:"ET TROJAN Win32/Spy.KeyLogger.ES CnC Response"; flow:to_server,established; content:"GET"; http_method; content:"Host|3a 20|"; http_header; depth:10; content:"User-Agent|3a 20|Mozilla/3.0 (compatible|3b 20|Indy Library|29 0d 0a|"; fast_pattern:5,20; http_header; depth:17; content:"Accept|3a 20|*/*|0d 0a|Accept-Language|3a 20|en-us|0d 0a|Cache-Control|3a 20|no-cache|0d 0a 0d 0a|Referer|3a 20|"; http_header; p','alert tcp $HOME_NET any -> $EXTERNAL_NET $HTTP_PORTS (msg:"ET TROJAN MINE20 Gemini Botnet User-Agent (plugins-qc)"; flow:to_server,established; content:"User-Agent|3a| gated by DNS|0d 0a|"; http_header; fast_pattern:only; metadata: former_category TROJAN; reference:md5,8617ae72368e8ab54b05bb52f3c922ca; classtype:trojan-activity; sid:2028379; rev:3; metadata:affected_product Windows_XP_Vista_7_8_10_Server_32_64_Bit, attack_target Client_Endpoint, deployment Perimeter, tag Gemini, tag Gh0st, tag RAT, signature_severity Critical, created_at 2019_03_15, malware_family Gh0st, malware_family InfernoBot, performance_impact Low, updated_at 2019_09_28;)']

scorer = score.BleurtScorer(checkpoint)
scores = scorer.score(references, candidates)
print(scores)

INFO:tensorflow:Reading checkpoint /content/bleurt/bleurt/test_checkpoint.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint bert_custom
INFO:tensorflow:Performs basic checks...
INFO:tensorflow:... name:bert_custom
INFO:tensorflow:... vocab_file:vocab.txt
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... do_lower_case:True
INFO:tensorflow:... max_seq_length:128
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Loading model...
INFO:tensorflow:Loading model.
INFO:tensorflow:Done.
INFO:tensorflow:BLEURT initialized.
[-0.5342968106269836, -0.042207177728414536, 0.026846710592508316]


In [ ]:
# with open("/content/drive/MyDrive/emerging-trojan.txt", 'r') as f:
#     lines = f.read().strip().split(';)')
#     print(lines)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:


# references = [[]]
# with open("/content/emerging-trojan.txt", 'r') as f:
#     lines = f.read().strip().split(';)')
#     print(lines)
#     #first_line = f.readline().strip()
#     for i, line in enumerate(lines):
#     #references[i].append()
#         references[i].append(line.strip().split(' '))

# generation=[]
# with open('snort_genrule_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())) as f:
#     generation.append(line.split() for line in f.readlines())
# score = sentence_bleu(references, generation)
# print(score)

##***BERT***

In [ ]:
!pip install bert_score
!pip install transformers

     |████████████████████████████████| 61kB 5.2MB/s 


In [ ]:
from bert_score import score
P, R, F1 = score(candidates, references, lang='others', verbose=True)


calculating scores...
computing bert embedding.



computing greedy matching.



done in 0.12 seconds, 25.48 sentences/sec


In [ ]:
print(F1)

tensor([0.7457, 0.7424, 0.8020])


##ROUGE

In [ ]:
!pip install rouge
from rouge import Rouge

In [ ]:
rouge = Rouge()
scores = rouge.get_scores(references, candidates)
print(scores)

[{'rouge-1': {'f': 0.28124999658203126, 'p': 0.18, 'r': 0.6428571428571429}, 'rouge-2': {'f': 0.19354838378251824, 'p': 0.12244897959183673, 'r': 0.46153846153846156}, 'rouge-l': {'f': 0.2903225771488034, 'p': 0.1875, 'r': 0.6428571428571429}}, {'rouge-1': {'f': 0.24657533767686257, 'p': 0.3103448275862069, 'r': 0.20454545454545456}, 'rouge-2': {'f': 0.1690140797302124, 'p': 0.21428571428571427, 'r': 0.13953488372093023}, 'rouge-l': {'f': 0.2608695603444655, 'p': 0.3103448275862069, 'r': 0.225}}, {'rouge-1': {'f': 0.3225806402589896, 'p': 0.375, 'r': 0.2830188679245283}, 'rouge-2': {'f': 0.2197802148822607, 'p': 0.2564102564102564, 'r': 0.19230769230769232}, 'rouge-l': {'f': 0.3370786467365232, 'p': 0.375, 'r': 0.30612244897959184}}]
